In [2]:
import cv2
import numpy as np
import os

In [3]:
image_dir = 'archive2/deepfashion2/images'

In [4]:
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Preprocess all images
for image_file in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_file)
    processed_image = preprocess_image(image_path)
    # Save or use the processed_image as needed

In [5]:
import pandas as pd

In [6]:
csv_file = 'archive2/deepfashion2/styles.csv'

In [7]:
annotations_df = pd.read_csv(csv_file, delimiter=';') 


In [8]:
# Create lists to store image IDs, bounding boxes, and labels
image_ids = []
bounding_boxes = []
labels = []

In [9]:
# Check if required columns exist
required_columns = ['image_id', 'bbox_xmin', 'bbox_ymin', 'bbox_xmax', 'bbox_ymax', 'category']
if all(col in annotations_df.columns for col in required_columns):
    # Iterate over the rows in the DataFrame
    for _, row in annotations_df.iterrows():
        try:
            # Extract image ID, bounding box, and label
            image_id = row['image_id']
            bbox = [row['bbox_xmin'], row['bbox_ymin'], row['bbox_xmax'], row['bbox_ymax']]
            label = row['category']

            # Append the data to respective lists
            image_ids.append(image_id)
            bounding_boxes.append(bbox)
            labels.append(label)
        except KeyError as e:
            print(f"Missing data in row {_}: {e}")
else:
    print("Error: Missing required columns in the CSV file.")

# Example of how to use the parsed data
for i in range(len(image_ids)):
    print(f"Image ID: {image_ids[i]}")
    print(f"Bounding Box: {bounding_boxes[i]}")
    print(f"Label: {labels[i]}")

Error: Missing required columns in the CSV file.


In [10]:
import tensorflow as tf

In [11]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalize to [0, 1]
    return image

def load_data(image_paths, labels):
    images = [load_image(p) for p in image_paths]
    labels = tf.convert_to_tensor(labels)
    return tf.data.Dataset.from_tensor_slices((images, labels))

In [12]:
import os
print(os.path.exists('archive2/deepfashion2/images'))


True


In [13]:
import pandas as pd

# Load the CSV file
annotations_df = pd.read_csv(csv_file, on_bad_lines='skip')

# Print column names
print("Columns in the CSV file:", annotations_df.columns)


Columns in the CSV file: Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName'],
      dtype='object')


In [14]:
import pandas as pd
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [15]:
def load_data(image_dir, csv_file, image_size=(224, 224)):
    # Load annotations from CSV
    annotations_df = pd.read_csv(csv_file, on_bad_lines='skip')
    
    # Print columns to find the correct names
    print("Columns in the CSV file:", annotations_df.columns)
    
    images = []
    labels = []
    
    # Iterate over the DataFrame
    for _, row in annotations_df.iterrows():
        # Construct the image filename based on the 'id' column
        image_filename = f"{row['id']}.jpg"  # Adjust if your image filenames have a different extension or format
        image_path = os.path.join(image_dir, image_filename)  # Construct full path
        
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Unable to load image at {image_path}")
            continue
        
        # Resize image to the desired size
        image = cv2.resize(image, image_size)
        
        # Use a relevant column for the label; you might choose 'masterCategory', 'subCategory', etc.
        label = row['masterCategory']  # Adjust as needed
        
        # Append image and label
        images.append(image)
        labels.append(label)
    
    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels


In [16]:
image_dir = '/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/images'
csv_file = '/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/styles.csv'

In [17]:
images, labels = load_data(image_dir, csv_file)

Columns in the CSV file: Index(['id', 'gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName'],
      dtype='object')


[ WARN:0@42.917] global loadsave.cpp:241 findDecoder imread_('/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/images/39403.jpg'): can't open/read file: check file path/integrity


[ WARN:0@46.701] global loadsave.cpp:241 findDecoder imread_('/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/images/39410.jpg'): can't open/read file: check file path/integrity


[ WARN:0@53.513] global loadsave.cpp:241 findDecoder imread_('/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/images/39401.jpg'): can't open/read file: check file path/integrity


[ WARN:0@55.507] global loadsave.cpp:241 findDecoder imread_('/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/images/39425.jpg'): can't open/read file: check file path/integrity


[ WARN:0@57.564] global loadsave.cpp:241 findDecoder imread_('/Users/lingesh_08/Desktop/VS CODE/archive2/deepfashion2/images/12347.jpg'): can't open/read file: check file path/integrity


In [18]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [19]:
from tensorflow.keras.utils import to_categorical

In [20]:
# Normalize images
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0